In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd
import seaborn as sns; print(sns.__version__)

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

In [ ]:
%%javascript
// needed to access notebook path later
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

In [ ]:
import os

def getAddInfo():
    import datetime
    # Get the current date and time in the format YYYY-MM-DD HH:MM:SS
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Get the name of the current jupyter notebook
    #notebook_name = os.path.basename(os.environ.get("NOTEBOOK_PATH", ""))
    notebook_name = NOTEBOOK_FULL_PATH
    #print(notebook_name)
    add_ttle = f"{now}\n{notebook_name}"
    return add_ttle

def addTitleInfo(axs):
    add_ttle = getAddInfo()
    # Add a linebreak and the date, time, and notebook name to the ylabel of each axis
    if axs is not None:
        if isinstance(axs, (list, np.ndarray)): # If axs is a list or an array of axes
            for ax in axs:
                title = ax.get_title()
                new_title = f"{title}\n{add_ttle}"
                ax.set_title(new_title)
        else: # If axs is a single axis
            title = axs.get_title()
            new_title = f"{title}\n{add_ttle}"
            axs.set_title(new_title)
    else:
        title = plt.gcf()._suptitle.get_text()
        new_title = f"{title}\n{add_ttle}"
        plt.suptitle(new_title)

In [ ]:

from config2 import path_data
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from behav_proc import *

import numpy as np
#from collections import OrderedDict as odict
from config2 import envcode2env
from pingouin import ttest

from base2 import radius_cursor
radius_cursor

In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz.pkl.zip')
print(fnf)
df_all_multi_tsz = pd.read_pickle(fnf)
#dfc_multi_tsz = pd.   read_pickle(pjoin(path_data,'df_all_multi_tsz_trunc_q=0.05.pkl.zip'))
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe"').copy()
df_orig = df
#del df_all_multi_tsz

df['env'] = df['environment'].apply(lambda x: envcode2env[x])
df['feedback_deg'] = df['feedback'] / np.pi * 180

def f(x):    
    if x > np.pi:
        x -= 2*np.pi
    elif x < -np.pi:
        x += 2*np.pi
    return x
df['error'] = df['error'].apply(f)

df['error_deg'] = df['error'] / np.pi * 180 

def f(x):    
    if x > np.pi:
        x -= 2*np.pi
    elif x < -np.pi:
        x += 2*np.pi
    return x
df['prev_error'] = df['prev_error'].apply(f)


In [ ]:
# estimate error at second halfs of init stage
qs_initstage = 'pert_stage_wb.abs() < 1e-10'
df_init = df.query(qs_initstage + ' and trialwb >= 10')
grp = df_init.groupby(['subject','pert_stage'])
#display(grp.size())
#df_init['feedback_deg'].min(), df_init['feedback_deg'].max()

grp['error_deg'].std()

stds = df_init.groupby(['subject'])['error_deg'].std()#.std()
mestd = stds.mean()
print(mestd, stds.std() )

stds = stds.to_frame().reset_index().rename(columns={'error_deg':'error_deg_initstd'})
df_wthr = df.merge(stds, on='subject')
#df_wthr

In [ ]:
df_all_multi_tsz_whtr = df_all_multi_tsz.merge(stds, on='subject')
df_all_multi_tsz_whtr_ = df_all_multi_tsz_whtr.\
    query('trial_group_col_calc == "trialwe"')

df_all_multi_tsz_whtr_['env'] = df_all_multi_tsz_whtr_['environment'].apply(lambda x: envcode2env[x])

In [ ]:
shiftszs = df_all_multi_tsz_whtr_['trial_shift_size'].unique()
shiftszs

In [ ]:
NN=5; st=0.4 #maxmult=1.2
#thrs =  np.linspace(mestd/NN, mestd, NN); print(mestd, thrs)
mults = st + (1-st) * np.arange(NN) / ( (NN - 1) )
mults = [0] + list(mults)
#thrs = mestd * mults
mults#, thrs

In [ ]:
# pooled random vs stable
dfs = []
ttrs = []
#for thri, thr in enumerate(mults):
for shiftsz in shiftszs:
    for multi, mult in enumerate(mults):
        # prev_error is not enough, it does not respect shift size
        df_ = df_all_multi_tsz_whtr_.query('trial_shift_size == @shiftsz').copy()
        df_ = df_[df_.error_deg.shift(shiftsz).abs() >= \
                df_['error_deg_initstd'] * mults[multi] ]
        #df_ = df_.query('error_deg.shift(1).abs() >= @thr')
        #thr_s = '{:.3f}'.format(thr)
        #thr_s = f'{maxmult}*mestd/{ NN - (thri)}'
        #thr_s = f'mestd/{ NN - (thri)}'
        thr_s = f'mestd*{mults[multi]}'
        df_['thr'] = thr_s
        dfs += [df_]
        print(thr_s, len(df_))
dfall_notclean = pd.concat(dfs)
# ttrs = pd.concat(ttrs)
# ttrs = ttrs.rename(columns={'p-val':'pval'})

from behav_proc import truncateDf
dfall_mshsz  = truncateDf(dfall_notclean, 'err_sens', q=0.0, infnan_handling='discard', 
                   cols_uniqify = ['subject','environment','thr','trial_shift_size'])


In [ ]:
dfc_multi_tsz = dfall_mshsz.query('thr == "mestd*1.0"')
#dfc_multi_tsz = dfall_mshsz.query('thr == "mestd*0"')

drs = dfc_multi_tsz['dist_rad_from_prevtgt'].unique();
drs_f = np.array( list(map( float, drs) ) )
drs = list(sorted(drs,key = lambda x: float(x)))

#dfc_s = dfc[dfc['trial_group_col_calc'] == 'trialwtgt_we']
# TODO: should I restrict to one env?
dfc_s = dfc_multi_tsz.query('trial_group_col_calc == "trialwe"')
grp = dfc_s.groupby(['trial_shift_size', 'dist_rad_from_prevtgt2'])
dts = dfc_multi_tsz['trial_shift_size'].unique()
nt = len(dts)

drpairs = [(drs[i],drs[i+1] ) for i in range(3)]
drpairs += [(drs[0],drs[i] ) for i in range(2,4)]
drpairs += [(drs[1],drs[3] )]
print(drpairs)

dtpairs = [(dts[i],dts[i+1] ) for i in range(3)]
dtpairs += [(dts[0],dts[i] ) for i in range(2,4)]
dtpairs += [(dts[1],dts[3] )]
dtpairs

In [ ]:
def getPvals(dftmp,fitcol,pairs):
    from scipy.stats import ttest_ind
    pvalues = []
    for drp in pairs:
        if isinstance(drp[0],str):
            vs1 = dftmp.query(f'{fitcol} == "{drp[0]}"')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == "{drp[1]}"')['err_sens']
        else:
            vs1 = dftmp.query(f'{fitcol} == {drp[0]}')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == {drp[1]}')['err_sens']
        #ttr = ttest_ind(vs1,vs2)
        ttr = ttest_ind(vs1,vs2, alternative='greater')
        pvalues += [ttr.pvalue]
        
        print(drp, ttr.pvalue)
    # pvalues = [
    #     sci_stats.mannwhitneyu(robots, flight, alternative="two-sided").pvalue,
    #     sci_stats.mannwhitneyu(flight, sound, alternative="two-sided").pvalue,
    #     sci_stats.mannwhitneyu(robots, sound, alternative="two-sided").pvalue
    # ]

    # pvalues
    # [0.00013485140468088997, 0.2557331102364572, 0.00022985464929005115]

    # Transform each p-value to "p=" in scientific notation
    formatted_pvalues = [f'p={pvalue:.2e}' for pvalue in pvalues]
    return pvalues, formatted_pvalues

# make polynomial fits
def plotPolys(ax, dftmp, fitcol, degs=range(2,6), mean=1):
    if mean:
        me = dftmp.groupby(fitcol).median(numeric_only=1).reset_index()
        dftmp = me
    dftmp[fitcol] = pd.to_numeric(dftmp[fitcol] )
    esv, dv = dftmp[['err_sens',fitcol]]._values.T
    print(np.min(dv),dv,dv-np.min(dv),esv)
    #pr = np.polyfit(esv,dv,2)
    from numpy.linalg import LinAlgError
    dvu = np.unique(dv)
    dvu = np.array( list(sorted(dvu)) )
    print(dvu)
    for deg in degs:
        try:
            pr = np.polyfit(dv-np.min(dv),esv-np.min(esv),deg)        
        except (SystemError,LinAlgError):
            print(f'Failed deg={deg}')
            print(dv,esv, np.std(dv))
            continue
            
        poly = np.poly1d(pr)
        #if len(degs) > 1:
        if mean:
            lbl = f'polynomial fit of means deg={deg}'
        else:
            lbl = f'polynomial fit deg={deg}'
        #else:
        #    lbl = None
        esv2 = poly(dvu-np.min(dvu)) + np.min(esv)
        print(dvu-np.min(dvu), esv)
        ax.plot(range(len(dvu)) , poly(dvu-np.min(dvu)) + np.min(esv), 
                label=lbl, c='grey', lw=0.85 )
    return pr
    #ax.legend(loc='lower right')

#plotPolys(ax,dftmp,fitcol,mean=0)
ax = plt.gca()
plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)

In [ ]:
# not pooled
#%debug

hue = 'env'
env2color = {'stable':'lightblue', 'random':'orange'}
aspect=1.3
#rng = [2]
#rng = [2,3,4,5,6,7]
rng = [2]
ylim = [-2,3]
meanfit=1
bnqs = [  ('grid_line_violin_spatial','Spatial distance [deg]',
           drs,drpairs,'dist_rad_from_prevtgt2', 
           f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'),
        ( 'grid_line_violin_temporal','Temporal distance [number of trials]',
         dts,dtpairs,'trial_shift_size',
         'trial_group_col_calc == "trialwe" and dist_rad_from_prevtgt2 == "0.00"')]

outfiles = []
for env_type in ['stable','random']:
    color = env2color[env_type]
    for bn,xlab,order,pairs,fitcol,qs in bnqs:
        #qs0 = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]}'
        #qs = qs + ' and trial_shift_size == 1'
        if env_type in ['stable', 'random']:
            qs += ' and env == @env_type'
            hue = None
            
        dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','environment',fitcol])
        dftmp0 = dftmp0.mean(numeric_only = True).reset_index()

        #dftmp0 = dfc_multi_tsz.query(qs)
        #fitcol = 'dist_rad_from_prevtgt2'
        fg = sns.catplot(kind='violin', data=dftmp0, y='err_sens', 
                         hue=hue, x=fitcol, order = order,
                          color=color, aspect=aspect, cut=True)
        #dftmp = dfc_multi_tsz.query(qs).copy()
        dftmp = dftmp0.copy()
        pvalues, formatted_pvalues = getPvals(dftmp,fitcol, pairs)


        for ax in fg.axes.flatten():
            ax.axhline(0,ls=':',c='r')
            ax.set_ylabel('Error sensitivity', fontdict={'fontsize':12})
            ax.set_xlabel(xlab, fontdict={'fontsize':12} )
            if bn == 'grid_line_violin_spatial':
                ax.set_xticklabels(['same target', 45, 90, 135])
            ax.set_ylim(ylim)
            ax.set_title(f'{env_type} environment', y = 0.967,
                        fontdict={'fontsize':15, 'fontweight':'bold'})


        from statannotations.Annotator import Annotator    
        annotator = Annotator(ax,pairs, data=dftmp0, x=fitcol, y='err_sens',
                             plot='violinplot', order=order)
        #annotator.set_custom_annotations(formatted_pvalues)
        annotator.set_pvalues(pvalues)
        annotator.annotate()

        #ax.set_ylim(-1,1.5)

        print('            ',env_type)
        plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)
        #bn = 'grid_line_violin_spatial'
        fnfig = pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}')
        plt.savefig(fnfig + '.svg')
        plt.savefig(fnfig + '.pdf')
        outfiles += [fnfig + '.pdf']


In [ ]:
import pingouin as pg

In [ ]:
dftmp0

In [ ]:
# how entire generalization lines are related (maybe useless)
print(getAddInfo())
for bn,xlab,order,pairs,fitcol,qs in bnqs:
    dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','env',fitcol])
    dftmp0 = dftmp0.mean(numeric_only = True).reset_index()    
    
    print(bn)
    ttrssig,ttrs = comparePairs(dftmp0, 'err_sens', 'env', alt=['greater','less'],
                               paired=True)
    display(ttrssig)
    #dftmp0.query('env == "stable"')['err_sens'], dftmp0.query('env == "random"')['err_sens']
    #break

In [ ]:
import warnings

print(getAddInfo())
corrs = []
for bn,xlab,order,pairs,fitcol,qs in bnqs:
    dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','env',fitcol])
    dftmp0 = dftmp0.mean(numeric_only = True).reset_index()    
    dftmp0[fitcol] = dftmp0[fitcol].astype(float) #now, not earlier!
    
    print(bn)
    
    def f(df_):
    #print(df_['err_sens'],df_[fitcol])
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            r = pg.corr(df_['err_sens'],df_[fitcol], method='pearson')
        return r
    r = dftmp0.groupby(['subject','env']).apply(f)

    coors_me = r.reset_index().groupby(['subject','env']).mean(numeric_only=1)
    coors_me['fitcol'] = fitcol
    corrs += [coors_me.reset_index()]
    for env in ['stable','random']:
        print(env)
        coors_me_ = coors_me.query('env == @env')
        corrs_sig_nonz = compare0(coors_me_, 'r').query('pval < 0.05')
        if len(corrs_sig_nonz):
            display(corrs_sig_nonz)
        else:
            print('   NOOOO')
    #dftmp0.query('env == "stable"')['err_sens'], dftmp0.query('env == "random"')['err_sens']
    #break
corrs = pd.concat(corrs, ignore_index=1)

In [ ]:
corrs

In [ ]:
def f(df_):
    #x = df_.query('env == "stable"')['r']
    #y = df_.query('env == "random"')['r']
    ttsig,tt = comparePairs( df_, 'r', 'env', alt=['greater','less'],
                               paired=True)
    return tt
dfr = corrs.groupby(['fitcol']).apply(f)

print(getAddInfo())
dfr.query('pval < 0.05')[['pval','pooled','ttstr']]

In [ ]:
dfr

In [ ]:
coors_me

In [ ]:
fitcol

In [ ]:
import subprocess

In [ ]:
input_files

In [ ]:
import shlex

In [ ]:
import subprocess as sp
input_files = outfiles

output_file = os.path.join(path_fig, "merged.pdf")

# Build the command to merge the PDF files using pdfunite
# #command = ["pdfunite"] + input_files + [output_file]
# command = ["pdfjam"] + input_files + ['--nup 2x2 --landscape --outfile'] + [output_file]

# #command = ' '.join(command)
# command = [command[0], ' '.join(command[1:])]

# # Run the command using subprocess.run()
# result = subprocess.run(command, capture_output=True, text=True)


# Build the command to arrange the PDF files using pdfjam
command = f"pdfjam {' '.join(input_files)} --nup 2x2 --suffix 4up"

# Split the command into a list of arguments using shlex.split()
args = shlex.split(command)

# Run the command using subprocess.run()
result = subprocess.run(args, capture_output=True, text=True)

# Check the return code and output of the command
if result.returncode == 0:
    print(f"Successfully merged {len(input_files)} ")
else:
    print(f"Failed to merge PDF files: {result.stderr}")

In [ ]:
result.stdout

In [ ]:
command

In [ ]:
import subprocess as sp

def create_stacked_pdf():
  """Creates a stacked PDF from the four input PDF files."""
  command = 'for env_type in random stable; do' \
            '  for bn in grid_line_violin_spatial grid_line_violin_temporal; do' \
            '    pdfunite nopool_${env_type}_${bn}.pdf nopool_${env_type}_${bn}.pdf >> stacked_pdf.pdf' \
            '  done' \
            'done'
  subprocess.call(command, shell=True)

create_stacked_pdf()